# Calculate S/T/Y ratio at position 0 & Scale PSPA

## Setup

In [ ]:
!pip install git+https://github.com/sky1ove/katlas.git -q

In [ ]:
from fastbook import *
import seaborn as sns
from katlas.core import *
from scipy.stats import spearmanr, pearsonr

from func import *

In [ ]:
sns.set(rc={"figure.dpi":200, 'savefig.dpi':200})
sns.set_context('notebook')
sns.set_style("ticks")

## Calculate s,t,y at position 0 using the PSPA raw data

In [ ]:
raw = Data.get_stack_raw().set_index('kinase')

In [ ]:
# Get S & T columns 
S_col = raw.columns[raw.columns.str.contains('S')]
T_col = raw.columns[raw.columns.str.contains('T')]

In [ ]:
# Get the sum of S & T columns respectively
S_sum = raw[S_col].sum(1)
T_sum = raw[T_col].sum(1)

In [ ]:
# Remove the background values of S and T
S_ctrl = 0.75*S_sum - 0.25*T_sum
T_ctrl = 0.75*T_sum - 0.25*S_sum

In [ ]:
# Combine the two above to a dataframe
ST = S_ctrl.reset_index(name='S_ctrl').merge(T_ctrl.reset_index(name='T_ctrl')).set_index('kinase')

In [ ]:
# Get S & T value at position 0
S0 = ST['S_ctrl']/ST.max(1)
T0 = ST['T_ctrl']/ST.max(1)

In [ ]:
# Combine the two above to a dataframe
ST0 = S0.reset_index(name='s').merge(T0.reset_index(name='t')).set_index('kinase')

Validate

In [ ]:
st_ref = pd.read_csv('raw/st_position0.csv',index_col=0)

In [ ]:
st_ref = st_ref.reindex(index=ST0.index)

In [ ]:
(st_ref == ST0.round(4)).value_counts()

s     t   
True  True    303
dtype: int64

## Add s,t,y at position 0 to the norm PSPA data

In [ ]:
# at y at position 0
ST0['y'] = 0.0

In [ ]:
# change the column name of sty so that it becomes 0S,0T or 0Y
ST0.columns  = "0" + ST0.columns 

In [ ]:
# Round it to 4 following Lew's paper's format
ST0 = ST0.round(4)

In [ ]:
# get norm PSPA data
norm = Data.get_stack_norm().set_index('kinase')

In [ ]:
# Add sty position 0 info to norm PSPA
norm = norm.merge(ST0,left_index=True, right_index=True)

In [ ]:
# save, this file is for calculating the score on pplus
norm.to_parquet('stack_norm.parquet')

In [ ]:
norm=pd.read_parquet('stack_norm.parquet')

In [ ]:
norm

,-5P,-5G,-5A,-5C,-5S,-5T,-5V,-5I,-5L,-5M,-5F,-5Y,-5W,-5H,-5K,-5R,-5Q,-5N,-5D,-5E,-5s,-5t,-5y,-4P,-4G,-4A,-4C,-4S,-4T,-4V,-4I,-4L,-4M,-4F,-4Y,-4W,-4H,-4K,-4R,-4Q,-4N,-4D,-4E,-4s,-4t,-4y,-3P,-3G,-3A,-3C,-3S,-3T,-3V,-3I,-3L,-3M,-3F,-3Y,-3W,-3H,-3K,-3R,-3Q,-3N,-3D,-3E,-3s,-3t,-3y,-2P,-2G,-2A,-2C,-2S,-2T,-2V,-2I,-2L,-2M,-2F,-2Y,-2W,-2H,-2K,-2R,-2Q,-2N,-2D,-2E,-2s,-2t,-2y,-1P,-1G,-1A,-1C,-1S,-1T,-1V,-1I,-1L,-1M,-1F,-1Y,-1W,-1H,-1K,-1R,-1Q,-1N,-1D,-1E,-1s,-1t,-1y,1P,1G,1A,1C,1S,1T,1V,1I,1L,1M,1F,1Y,1W,1H,1K,1R,1Q,1N,1D,1E,1s,1t,1y,2P,2G,2A,2C,2S,2T,2V,2I,2L,2M,2F,2Y,2W,2H,2K,2R,2Q,2N,2D,2E,2s,2t,2y,3P,3G,3A,3C,3S,3T,3V,3I,3L,3M,3F,3Y,3W,3H,3K,3R,3Q,3N,3D,3E,3s,3t,3y,4P,4G,4A,4C,4S,4T,4V,4I,4L,4M,4F,4Y,4W,4H,4K,4R,4Q,4N,4D,4E,4s,4t,4y,0s,0t,0y
kinase,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAK1,0.0720,0.0245,0.0284,0.0456,0.0425,0.0425,0.0951,0.1554,0.0993,0.0864,0.0425,0.0952,0.0315,0.0331,0.0262,0.0956,0.0560,0.0275,0.0160,0.0153,0.0201,0.0201,0.0611,0.0534,0.0642,0.0706,0.0560,0.0619,0.0619,0.0619,0.0621,0.0742,0.0693,0.0520,0.0534,0.0403,0.0514,0.0809,0.0715,0.0627,0.0429,0.0332,0.0560,0.0332,0.0332,0.0339,0.1084,0.0512,0.1119,0.0655,0.0527,0.0527,0.0641,0.0599,0.0527,0.0782,0.0487,0.0411,0.0425,0.0397,0.0517,0.0581,0.0695,0.0582,0.0272,0.0370,0.0303,0.0303,0.0274,0.0226,0.0283,0.0538,0.0588,0.0555,0.0555,0.0340,0.0378,0.0569,0.1166,0.0336,0.0399,0.0289,0.0567,0.0587,0.0555,0.1517,0.0808,0.0805,0.0636,0.0209,0.0209,0.0486,0.1136,0.0706,0.0385,0.0313,0.0545,0.0545,0.0392,0.0435,0.0654,0.0554,0.0545,0.0777,0.0440,0.0688,0.1229,0.0923,0.0397,0.0434,0.0161,0.0146,0.0121,0.0121,0.0178,0.0463,0.7216,0.0312,0.0223,0.0143,0.0143,0.0107,0.0102,0.0131,0.0124,0.0129,0.0143,0.0162,0.0160,0.0194,0.0249,0.0138,0.0170,0.0103,0.0098,0.0123,0.0123,0.0100,0.0527,0.0749,0.0750,0.0684,0.0552,0.0552,0.0542,0.0370,0.0414,0.0481,0.0486,0.0459,0.0572,0.0620,0.0577,0.0774,0.0771,0.0834,0.0552,0.0524,0.0409,0.0409,0.0410,0.0681,0.0923,0.0582,0.0742,0.0582,0.0582,0.0610,0.0388,0.0489,0.0437,0.0430,0.0533,0.0481,0.0674,0.0739,0.0901,0.0623,0.0735,0.0405,0.0371,0.0335,0.0335,0.0359,0.0628,0.0702,0.0646,0.0603,0.0560,0.0560,0.0422,0.0415,0.0461,0.0464,0.0523,0.0521,0.0826,0.0560,0.0831,0.0928,0.0635,0.0592,0.0389,0.0457,0.0251,0.0251,0.0270,0.1013,1.0000,0.0
ACVR2A,0.0415,0.0481,0.0584,0.0489,0.0578,0.0578,0.0598,0.0625,0.0596,0.0521,0.0600,0.0578,0.0803,0.0570,0.0510,0.0475,0.0430,0.0536,0.0888,0.0789,0.0783,0.0783,0.0760,0.0466,0.0549,0.0555,0.0551,0.0549,0.0549,0.0543,0.0526,0.0520,0.0576,0.0619,0.0586,0.0754,0.0551,0.0406,0.0457,0.0482,0.0501,0.1040,0.0869,0.0809,0.0809,0.0681,0.0502,0.0653,0.0537,0.0588,0.0516,0.0516,0.0546,0.0503,0.0479,0.0515,0.0516,0.0550,0.0591,0.0529,0.0355,0.0461,0.0508,0.0502,0.1050,0.1205,0.1205,0.1205,0.0975,0.0189,0.0294,0.0312,0.0726,0.0295,0.0295,0.0295,0.0267,0.0253,0.0234,0.0297,0.0293,0.0317,0.0287,0.0208,0.0297,0.0452,0.0398,0.2530,0.3077,0.0595,0.0595,0.0424,0.0398,0.0335,0.0484,0.1042,0.0630,0.0630,0.0650,0.0473,0.0750,0.0847,0.0688,0.0783,0.0816,0.0630,0.0313,0.0340,0.0529,0.0480,0.0724,0.0761,0.0807,0.0807,0.1065,0.0200,0.0338,0.0381,0.0748,0.0548,0.0548,0.0779,0.0691,0.0514,0.0586,0.0548,0.0571,0.0566,0.0412,0.0192,0.0245,0.0723,0.0375,0.0990,0.1888,0.2735,0.2735,0.1651,0.0582,0.0799,0.0660,0.0600,0.0582,0.0582,0.0588,0.0538,0.0397,0.0517,0.0578,0.0651,0.0578,0.0660,0.0473,0.0550,0.0591,0.0507,0.0728,0.0604,0.0650,0.0650,0.0516,0.0670,0.0592,0.0529,0.0481,0.0590,0.0590,0.0567,0.0554,0.0612,0.0589,0.0644,0.0654,0.0635,0.0590,0.0462,0.0387,0.0499,0.0524,0.0622,0.0870,0.0519,0.0519,0.0815,0.0758,0.0544,0.0498,0.0517,0.0563,0.0563,0.0516,0.0563,0.0512,0.0662,0.0523,0.0579,0.0800,0.0573,0.0527,0.0491,0.0616,0.0556,0.0640,0.0640,0.0703,0.0703,0.0589,0.9833,1.0000,0.0
ACVR2B,0.0533,0.0517,0.0566,0.0772,0.0533,0.0533,0.0543,0.0442,0.0471,0.0516,0.0604,0.0607,

## Scale each position to 1

In [ ]:
data = []
specificities=[]
for k in norm.index:
    df_k = get_one_kinase2(norm,k).T.fillna(0)
    scaled = df_k/df_k.sum()
    specificity = scaled.drop(columns=[0]).max().max()
    data.append(scaled.unstack().values)
    specificities.append(specificity)

In [ ]:
col = scaled.unstack().reset_index()
col['Position'] = col['position'].astype(str) + col['aa']

In [ ]:
scaled = pd.DataFrame(data,columns=col['Position'].tolist(),index=pspa.index)

scaled = scaled[norm.columns]

In [ ]:
scaled

,-5P,-5G,-5A,-5C,-5S,-5T,-5V,-5I,-5L,-5M,-5F,-5Y,-5W,-5H,-5K,-5R,-5Q,-5N,-5D,-5E,-5s,-5t,-5y,-4P,-4G,-4A,-4C,-4S,-4T,-4V,-4I,-4L,-4M,-4F,-4Y,-4W,-4H,-4K,-4R,-4Q,-4N,-4D,-4E,-4s,-4t,-4y,-3P,-3G,-3A,-3C,-3S,-3T,-3V,-3I,-3L,-3M,-3F,-3Y,-3W,-3H,-3K,-3R,-3Q,-3N,-3D,-3E,-3s,-3t,-3y,-2P,-2G,-2A,-2C,-2S,-2T,-2V,-2I,-2L,-2M,-2F,-2Y,-2W,-2H,-2K,-2R,-2Q,-2N,-2D,-2E,-2s,-2t,-2y,-1P,-1G,-1A,-1C,-1S,-1T,-1V,-1I,-1L,-1M,-1F,-1Y,-1W,-1H,-1K,-1R,-1Q,-1N,-1D,-1E,-1s,-1t,-1y,1P,1G,1A,1C,1S,1T,1V,1I,1L,1M,1F,1Y,1W,1H,1K,1R,1Q,1N,1D,1E,1s,1t,1y,2P,2G,2A,2C,2S,2T,2V,2I,2L,2M,2F,2Y,2W,2H,2K,2R,2Q,2N,2D,2E,2s,2t,2y,3P,3G,3A,3C,3S,3T,3V,3I,3L,3M,3F,3Y,3W,3H,3K,3R,3Q,3N,3D,3E,3s,3t,3y,4P,4G,4A,4C,4S,4T,4V,4I,4L,4M,4F,4Y,4W,4H,4K,4R,4Q,4N,4D,4E,4s,4t,4y,0s,0t,0y
kinase,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAK1,0.058446,0.019888,0.023054,0.037016,0.034500,0.034500,0.077198,0.126147,0.080607,0.070136,0.034500,0.077279,0.025570,0.026869,0.021268,0.077604,0.045458,0.022323,0.012988,0.012420,0.016316,0.016316,0.049598,0.041715,0.050152,0.055152,0.043747,0.048356,0.048356,0.048356,0.048512,0.057964,0.054136,0.040622,0.041715,0.031482,0.040153,0.063198,0.055855,0.048981,0.033513,0.025935,0.043747,0.025935,0.025935,0.026482,0.086100,0.040667,0.088880,0.052025,0.041859,0.041859,0.050913,0.047577,0.041859,0.062113,0.038681,0.032645,0.033757,0.031533,0.041064,0.046148,0.055203,0.046227,0.021604,0.029388,0.024067,0.024067,0.021763,0.017935,0.022459,0.042695,0.046663,0.044044,0.044044,0.026982,0.029998,0.045155,0.092532,0.026665,0.031664,0.022935,0.044996,0.046584,0.044044,0.120387,0.064122,0.063884,0.050472,0.016586,0.016586,0.038568,0.096068,0.059704,0.032558,0.026469,0.046089,0.046089,0.033150,0.036786,0.055307,0.046850,0.046089,0.065708,0.037209,0.058182,0.103932,0.078055,0.033573,0.036702,0.013615,0.012347,0.010233,0.010233,0.015053,0.042649,0.664702,0.028740,0.020542,0.013172,0.013172,0.009856,0.009396,0.012067,0.011422,0.011883,0.013172,0.014923,0.014738,0.017870,0.022937,0.012712,0.015660,0.009488,0.009027,0.011330,0.011330,0.009211,0.040482,0.057536,0.057613,0.052543,0.042403,0.042403,0.041635,0.028422,0.031802,0.036949,0.037333,0.035259,0.043939,0.047626,0.044323,0.059456,0.059226,0.064065,0.042403,0.040252,0.031418,0.031418,0.031495,0.052640,0.071346,0.044987,0.057355,0.044987,0.044987,0.047152,0.029991,0.037799,0.033779,0.033238,0.041200,0.037180,0.052099,0.057123,0.069645,0.048156,0.056814,0.031306,0.028677,0.025895,0.025895,0.027750,0.050260,0.056182,0.051701,0.048259,0.044818,0.044818,0.033774,0.033213,0.036895,0.037135,0.041857,0.041697,0.066106,0.044818,0.066507,0.074270,0.050820,0.047379,0.031132,0.036575,0.020088,0.020088,0.021609,0.091982,0.908018,0.0
ACVR2A,0.029707,0.034431,0.041804,0.035004,0.041374,0.041374,0.042806,0.044739,0.042663,0.037294,0.042949,0.041374,0.057480,0.040802,0.036507,0.034001,0.030780,0.038368,0.063565,0.056478,0.056049,0.056049,0.054402,0.033410,0.039360,0.039791,0.039504,0.039360,0.039360,0.038930,0.037711,0.037281,0.041296,0.044379,0.042013,0.054058,0.039504,0.029108,0.032765,0.034557,0.035919,0.074563,0.062303,0.058001,0.058001,0.048824,0.033451,0.043513,0.035783,0.039182,0.034384,0.034384,0.036383,0.033518,0.031918,0.034317,0.034384,0.036650,0.039382,0.035250,0.023656,0.030719,0.033851,0.033451,0.069967,0.080296,0.080296,0.080296,0.064970,0.014617,0.022738,0.024130,0.056148,0.022815,0.022815,0.022815,0.020650,0.019567,0.018097,0.022970,0.022660,0.024517,0.022196,0.016087,0.022970,0.034957,0.030781,0.195669,0.237974,0.046017,0.046017,0.032792,0.026565,0.022360,0.032305,0.069550,0.042050,0.042050,0.043385,0.031571,0.050060,0.056535,0.045922,0.052263,0.054465,0.042050,0.020892,0.022694,0.035309,0.032038,0.048325,0.050794,0.053865,0.053865,0.071085,0.010546,0.017823,0.020091,0.039443,0.028897,0.028897,0.041078,0.036437,0.027104,0.030901,0.028897,0.0301

For specificities

In [ ]:
speci = pd.DataFrame(specificities,columns=['max'],index=scaled.index)

In [ ]:
speci.to_csv('specificity_raw.csv')

Confirm the scaled

In [ ]:
for k in scaled.index:
    df_k = get_one_kinase2(scaled,k).T.fillna(0)
    break

In [ ]:
df_k.sum()

position
-5    1.0
-4    1.0
-3    1.0
-2    1.0
-1    1.0
 0    1.0
 1    1.0
 2    1.0
 3    1.0
 4    1.0
dtype: float64

In [ ]:
scaled.to_parquet('pspa_main.parquet')

## To capital
> Get freq that combined s,t,y to S,T,Y

In [ ]:
upper = scaled.copy()

In [ ]:
# List of suffixes
suffixes = ['S', 'T', 'Y']

for suffix in suffixes:
    for i in range(-5, 5):  # looping from -7 to 7
        if i == 0:  # Skip 0
            continue
        
        upper_col = f"{i}{suffix}"  # e.g., -7S
        lower_col = f"{i}{suffix.lower()}"  # e.g., -7s
        upper[upper_col] = upper[upper_col] + upper[lower_col]
        upper.drop(lower_col, axis=1,inplace=True)  # Drop the lowercase column after combining

In [ ]:
upper

,-5P,-5G,-5A,-5C,-5S,-5T,-5V,-5I,-5L,-5M,-5F,-5Y,-5W,-5H,-5K,-5R,-5Q,-5N,-5D,-5E,-4P,-4G,-4A,-4C,-4S,-4T,-4V,-4I,-4L,-4M,-4F,-4Y,-4W,-4H,-4K,-4R,-4Q,-4N,-4D,-4E,-3P,-3G,-3A,-3C,-3S,-3T,-3V,-3I,-3L,-3M,-3F,-3Y,-3W,-3H,-3K,-3R,-3Q,-3N,-3D,-3E,-2P,-2G,-2A,-2C,-2S,-2T,-2V,-2I,-2L,-2M,-2F,-2Y,-2W,-2H,-2K,-2R,-2Q,-2N,-2D,-2E,-1P,-1G,-1A,-1C,-1S,-1T,-1V,-1I,-1L,-1M,-1F,-1Y,-1W,-1H,-1K,-1R,-1Q,-1N,-1D,-1E,1P,1G,1A,1C,1S,1T,1V,1I,1L,1M,1F,1Y,1W,1H,1K,1R,1Q,1N,1D,1E,2P,2G,2A,2C,2S,2T,2V,2I,2L,2M,2F,2Y,2W,2H,2K,2R,2Q,2N,2D,2E,3P,3G,3A,3C,3S,3T,3V,3I,3L,3M,3F,3Y,3W,3H,3K,3R,3Q,3N,3D,3E,4P,4G,4A,4C,4S,4T,4V,4I,4L,4M,4F,4Y,4W,4H,4K,4R,4Q,4N,4D,4E,0s,0t,0y
kinase,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAK1,0.058446,0.019888,0.023054,0.037016,0.050816,0.050816,0.077198,0.126147,0.080607,0.070136,0.034500,0.126877,0.025570,0.026869,0.021268,0.077604,0.045458,0.022323,0.012988,0.012420,0.041715,0.050152,0.055152,0.043747,0.074291,0.074291,0.048356,0.048512,0.057964,0.054136,0.040622,0.068198,0.031482,0.040153,0.063198,0.055855,0.048981,0.033513,0.025935,0.043747,0.086100,0.040667,0.088880,0.052025,0.065925,0.065925,0.050913,0.047577,0.041859,0.062113,0.038681,0.054408,0.033757,0.031533,0.041064,0.046148,0.055203,0.046227,0.021604,0.029388,0.017935,0.022459,0.042695,0.046663,0.060630,0.060630,0.026982,0.029998,0.045155,0.092532,0.026665,0.070233,0.022935,0.044996,0.046584,0.044044,0.120387,0.064122,0.063884,0.050472,0.096068,0.059704,0.032558,0.026469,0.056321,0.056321,0.033150,0.036786,0.055307,0.046850,0.046089,0.080761,0.037209,0.058182,0.103932,0.078055,0.033573,0.036702,0.013615,0.012347,0.042649,0.664702,0.028740,0.020542,0.024503,0.024503,0.009856,0.009396,0.012067,0.011422,0.011883,0.022384,0.014923,0.014738,0.017870,0.022937,0.012712,0.015660,0.009488,0.009027,0.040482,0.057536,0.057613,0.052543,0.073821,0.073821,0.041635,0.028422,0.031802,0.036949,0.037333,0.066754,0.043939,0.047626,0.044323,0.059456,0.059226,0.064065,0.042403,0.040252,0.052640,0.071346,0.044987,0.057355,0.070882,0.070882,0.047152,0.029991,0.037799,0.033779,0.033238,0.068950,0.037180,0.052099,0.057123,0.069645,0.048156,0.056814,0.031306,0.028677,0.050260,0.056182,0.051701,0.048259,0.064906,0.064906,0.033774,0.033213,0.036895,0.037135,0.041857,0.063305,0.066106,0.044818,0.066507,0.074270,0.050820,0.047379,0.031132,0.036575,0.091982,0.908018,0.0
ACVR2A,0.029707,0.034431,0.041804,0.035004,0.097423,0.097423,0.042806,0.044739,0.042663,0.037294,0.042949,0.095777,0.057480,0.040802,0.036507,0.034001,0.030780,0.038368,0.063565,0.056478,0.033410,0.039360,0.039791,0.039504,0.097362,0.097362,0.038930,0.037711,0.037281,0.041296,0.044379,0.090837,0.054058,0.039504,0.029108,0.032765,0.034557,0.035919,0.074563,0.062303,0.033451,0.043513,0.035783,0.039182,0.114680,0.114680,0.036383,0.033518,0.031918,0.034317,0.034384,0.101619,0.039382,0.035250,0.023656,0.030719,0.033851,0.033451,0.069967,0.080296,0.014617,0.022738,0.024130,0.056148,0.068832,0.068832,0.022815,0.020650,0.019567,0.018097,0.022970,0.055452,0.024517,0.022196,0.016087,0.022970,0.034957,0.030781,0.195669,0.237974,0.026565,0.022360,0.032305,0.069550,0.095915,0.095915,0.043385,0.031571,0.050060,0.056535,0.045922,0.123348,0.054465,0.042050,0.020892,0.022694,0.035309,0.032038,0.048325,0.050794,0.010546,0.017823,0.020091,0.039443,0.173117,0.173117,0.041078,0.036437,0.027104,0.030901,0.028897,0.117169,0.029846,0.021725,0.010124,0.012919,0.038125,0.019774,0.052204,0.099557,0.042854,0.058832,0.048597,0.044179,0.090715,0.090715,0.043296,0.039614,0.029232,0.038068,0.042559,0.085929,0.042559,0.048597,0.034828,0.040498,0.043517,0.037332,0.053604,0.044474,0.049578,0.043806,0.039145,0.035593,0.082063,0.082063,0.041956,0.040995,0.045286,0.043584,0.047654,0.108702,0.046988,0.043658,0.034187,0.028637,0.036925,0.038775,0.046026,0.064378,0.055588,0.039894,0.036521,0.037914,0.092842,0.092842,0.037841,0.0

Confirm

In [ ]:
upper.columns[upper.columns.str.contains('S')]

Index(['-5S', '-4S', '-3S', '-2S', '-1S', '1S', '2S', '3S', '4S'], dtype='object')

In [ ]:
upper.columns[upper.columns.str.contains('s')]

Index(['0s'], dtype='object')

In [ ]:
upper.loc[:,upper.columns.str.contains('-5')].sum(1).sort_values()

kinase
YSK1     1.0
ICK      1.0
P38B     1.0
NIK      1.0
PAK3     1.0
        ... 
IKKE     1.0
TSSK1    1.0
TNIK     1.0
DMPK1    1.0
CK1D     1.0
Length: 303, dtype: float64

In [ ]:
upper = upper.rename(columns={'0s':'0S','0t':'0T','0y':'0Y'})

In [ ]:
upper

,-5P,-5G,-5A,-5C,-5S,-5T,-5V,-5I,-5L,-5M,-5F,-5Y,-5W,-5H,-5K,-5R,-5Q,-5N,-5D,-5E,-4P,-4G,-4A,-4C,-4S,-4T,-4V,-4I,-4L,-4M,-4F,-4Y,-4W,-4H,-4K,-4R,-4Q,-4N,-4D,-4E,-3P,-3G,-3A,-3C,-3S,-3T,-3V,-3I,-3L,-3M,-3F,-3Y,-3W,-3H,-3K,-3R,-3Q,-3N,-3D,-3E,-2P,-2G,-2A,-2C,-2S,-2T,-2V,-2I,-2L,-2M,-2F,-2Y,-2W,-2H,-2K,-2R,-2Q,-2N,-2D,-2E,-1P,-1G,-1A,-1C,-1S,-1T,-1V,-1I,-1L,-1M,-1F,-1Y,-1W,-1H,-1K,-1R,-1Q,-1N,-1D,-1E,1P,1G,1A,1C,1S,1T,1V,1I,1L,1M,1F,1Y,1W,1H,1K,1R,1Q,1N,1D,1E,2P,2G,2A,2C,2S,2T,2V,2I,2L,2M,2F,2Y,2W,2H,2K,2R,2Q,2N,2D,2E,3P,3G,3A,3C,3S,3T,3V,3I,3L,3M,3F,3Y,3W,3H,3K,3R,3Q,3N,3D,3E,4P,4G,4A,4C,4S,4T,4V,4I,4L,4M,4F,4Y,4W,4H,4K,4R,4Q,4N,4D,4E,0S,0T,0Y
kinase,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAK1,0.058446,0.019888,0.023054,0.037016,0.050816,0.050816,0.077198,0.126147,0.080607,0.070136,0.034500,0.126877,0.025570,0.026869,0.021268,0.077604,0.045458,0.022323,0.012988,0.012420,0.041715,0.050152,0.055152,0.043747,0.074291,0.074291,0.048356,0.048512,0.057964,0.054136,0.040622,0.068198,0.031482,0.040153,0.063198,0.055855,0.048981,0.033513,0.025935,0.043747,0.086100,0.040667,0.088880,0.052025,0.065925,0.065925,0.050913,0.047577,0.041859,0.062113,0.038681,0.054408,0.033757,0.031533,0.041064,0.046148,0.055203,0.046227,0.021604,0.029388,0.017935,0.022459,0.042695,0.046663,0.060630,0.060630,0.026982,0.029998,0.045155,0.092532,0.026665,0.070233,0.022935,0.044996,0.046584,0.044044,0.120387,0.064122,0.063884,0.050472,0.096068,0.059704,0.032558,0.026469,0.056321,0.056321,0.033150,0.036786,0.055307,0.046850,0.046089,0.080761,0.037209,0.058182,0.103932,0.078055,0.033573,0.036702,0.013615,0.012347,0.042649,0.664702,0.028740,0.020542,0.024503,0.024503,0.009856,0.009396,0.012067,0.011422,0.011883,0.022384,0.014923,0.014738,0.017870,0.022937,0.012712,0.015660,0.009488,0.009027,0.040482,0.057536,0.057613,0.052543,0.073821,0.073821,0.041635,0.028422,0.031802,0.036949,0.037333,0.066754,0.043939,0.047626,0.044323,0.059456,0.059226,0.064065,0.042403,0.040252,0.052640,0.071346,0.044987,0.057355,0.070882,0.070882,0.047152,0.029991,0.037799,0.033779,0.033238,0.068950,0.037180,0.052099,0.057123,0.069645,0.048156,0.056814,0.031306,0.028677,0.050260,0.056182,0.051701,0.048259,0.064906,0.064906,0.033774,0.033213,0.036895,0.037135,0.041857,0.063305,0.066106,0.044818,0.066507,0.074270,0.050820,0.047379,0.031132,0.036575,0.091982,0.908018,0.0
ACVR2A,0.029707,0.034431,0.041804,0.035004,0.097423,0.097423,0.042806,0.044739,0.042663,0.037294,0.042949,0.095777,0.057480,0.040802,0.036507,0.034001,0.030780,0.038368,0.063565,0.056478,0.033410,0.039360,0.039791,0.039504,0.097362,0.097362,0.038930,0.037711,0.037281,0.041296,0.044379,0.090837,0.054058,0.039504,0.029108,0.032765,0.034557,0.035919,0.074563,0.062303,0.033451,0.043513,0.035783,0.039182,0.114680,0.114680,0.036383,0.033518,0.031918,0.034317,0.034384,0.101619,0.039382,0.035250,0.023656,0.030719,0.033851,0.033451,0.069967,0.080296,0.014617,0.022738,0.024130,0.056148,0.068832,0.068832,0.022815,0.020650,0.019567,0.018097,0.022970,0.055452,0.024517,0.022196,0.016087,0.022970,0.034957,0.030781,0.195669,0.237974,0.026565,0.022360,0.032305,0.069550,0.095915,0.095915,0.043385,0.031571,0.050060,0.056535,0.045922,0.123348,0.054465,0.042050,0.020892,0.022694,0.035309,0.032038,0.048325,0.050794,0.010546,0.017823,0.020091,0.039443,0.173117,0.173117,0.041078,0.036437,0.027104,0.030901,0.028897,0.117169,0.029846,0.021725,0.010124,0.012919,0.038125,0.019774,0.052204,0.099557,0.042854,0.058832,0.048597,0.044179,0.090715,0.090715,0.043296,0.039614,0.029232,0.038068,0.042559,0.085929,0.042559,0.048597,0.034828,0.040498,0.043517,0.037332,0.053604,0.044474,0.049578,0.043806,0.039145,0.035593,0.082063,0.082063,0.041956,0.040995,0.045286,0.043584,0.047654,0.108702,0.046988,0.043658,0.034187,0.028637,0.036925,0.038775,0.046026,0.064378,0.055588,0.039894,0.036521,0.037914,0.092842,0.092842,0.037841,0.0

In [ ]:
upper.to_parquet('pspa_main_upper.parquet')